# Test IRS-Net on Test Data Using Pretrained Network

## Load Libraries

In [4]:
import matplotlib.pyplot as plt
from scipy.io import savemat
from scipy.io import loadmat


In [5]:
import numpy as np
import tensorflow as tf
import time

## System Parameters for Noise Limited Scenario

In [42]:
eta = 1
P_b = 10        #Watt or 40 dBm
P_i = 0         #Watt or 40 dBm
sigma_n = np.sqrt(3.98*10**-14)
M = 2           #Power Beam
N = 8           #of IRS elements

## Path Loss Model Parameters

In [43]:
alpha = 2.57
alpha_irs = 2.2;
Lc = 10**-3;

d_br = 20;
d_rs = 15;
d_sb = 25;
d_is = 15;
d_id = 30
d_rd = 15;
d_sd = 25;
d_ir = 15;

b_bs = Lc*d_sb**(-alpha);
b_is = Lc*d_is**(-alpha);
b_id = Lc*d_id**(-alpha);
b_sd = Lc*d_sd**(-alpha_irs);
b_rs = Lc*d_rs**(-alpha_irs);
b_rd = Lc*d_rd**(-alpha_irs);
b_ir = Lc*d_ir**(-alpha_irs);
b_br = Lc*d_br**(-alpha_irs);

## Load Pretained Weights
To load pretrained parameters,
i) Download pretrained parameters from this repository.
ii) Un-zip folder.
iii) Give directory path, where you unzipped pretrained weights, in folowing cell.


In [44]:
if M==2 and N==8:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M2N8Pi0'
elif M==4 and N==8:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M4N8Pi0'
elif M==8 and N==8:
    print(N,M)
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M8N8Pi0'
elif M==2 and N==16:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M2N16Pi0'
elif M==4 and N==16:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M4N16Pi0'
elif M==8 and N==16:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M8N16Pi0'
elif M==2 and N==32:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M2N32Pi0'
elif M==4 and N==32:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M4N32Pi0'
elif M==8 and N==32:
    trained_param_folder = r'C:\Users\Ahsann Mehmood\Desktop\IRS revision\Final revision\Noise_limited_scenario\M8N32Pi0'

W1 = loadmat(trained_param_folder+'/W1.mat')
W2 = loadmat(trained_param_folder+'/W2.mat')
W3 = loadmat(trained_param_folder+'/W3.mat')
W4 = loadmat(trained_param_folder+'/W4.mat')
W5 = loadmat(trained_param_folder+'/W5.mat')
W6 = loadmat(trained_param_folder+'/W6.mat')
    
b1 = loadmat(trained_param_folder+'/b1.mat')
b2 = loadmat(trained_param_folder+'/b2.mat')
b3 = loadmat(trained_param_folder+'/b3.mat')
b4 = loadmat(trained_param_folder+'/b4.mat')
b5 = loadmat(trained_param_folder+'/b5.mat')
b6 = loadmat(trained_param_folder+'/b6.mat')
trained_param_folder


'C:\\Users\\Ahsann Mehmood\\Desktop\\IRS revision\\Final revision\\Noise_limited_scenario\\M2N8Pi0'

In [45]:
print(W3['W3'].shape)
print(W6['W6'].shape)

(243, 243)
(18, 54)


## Make a Placeholder to feed feature vector.

In [46]:
 
X = tf.placeholder(tf.float64, shape=(2*(N*M+M+N+1),None))

## Forward Propagation

In [47]:
Z1 = tf.add(tf.matmul(W1['W1'],X),b1['b1'])                     # Linear Layer
mean, variance = tf.nn.moments(Z1, axes= [1] ,keepdims=True)
Z1_BN = tf.nn.batch_normalization(Z1, mean=mean, variance= variance, scale=1.0, offset= None, variance_epsilon=10**-5) # Batch Normalization Layer
A1 = tf.nn.relu(Z1_BN)      # Relu Layer
 
Z2 = tf.add(tf.matmul(W2['W2'],A1),b2['b2'])                     # Linear Layer
mean, variance = tf.nn.moments(Z2, axes= [1] ,keepdims=True)
Z2_BN = tf.nn.batch_normalization(Z2, mean=mean, variance= variance, scale=1.0, offset= None, variance_epsilon=10**-5) # Batch Normalization Layer
A2 = tf.nn.relu(Z2)
 
Z3 = tf.add(tf.matmul(W3['W3'],A2),b3['b3'])                     # Linear Layer
mean, variance = tf.nn.moments(Z3, axes= [1] ,keepdims=True)
Z3_BN = tf.nn.batch_normalization(Z3, mean=mean, variance= variance, scale=1, offset= None, variance_epsilon=10**-5) # Batch Normalization Layer
A3 = tf.nn.relu(Z3)

Z4 = tf.add(tf.matmul(W4['W4'],A3),b4['b4'])                     # Linear Layer
mean, variance = tf.nn.moments(Z4, axes= [1],keepdims=True)
Z4_BN = tf.nn.batch_normalization(Z4, mean=mean, variance= variance, scale=1, offset= None, variance_epsilon=10**-5) # Batch Normalization Layer
A4 = tf.nn.relu(Z4) 

Z5 = tf.add(tf.matmul(W5['W5'],A4),b5['b5'])                     # Linear Layer
mean, variance = tf.nn.moments(Z5, axes= [1],keepdims=True)
Z5_BN = tf.nn.batch_normalization(Z5, mean=mean, variance= variance, scale=1, offset= None, variance_epsilon=10**-5) # Batch Normalization Layer
A5 = tf.nn.leaky_relu(Z5_BN,0.) 

Z6 = tf.add(tf.matmul(W6['W6'],A5),b6['b6'])                     # Linear Layer
theta1 = tf.nn.sigmoid(Z6)

## Compute Cost

In [48]:
real = tf.zeros([N,tf.shape(X)[1]],dtype=tf.float64)
log_base = tf.constant(2,dtype=tf.float64)
etta = 1 

In [49]:
theta_et1, theta_it1,tau,lemda = tf.split(theta1, num_or_size_splits= [N,N,1,1], axis = 0)
theta_et = tf.exp(tf.complex(real,theta_et1*2*np.pi))
theta_it = tf.exp(tf.complex(real,theta_it1*2*np.pi))

g_theta_et1 = tf.reshape(tf.complex(X[0:N*M,:],X[N*M:2*N*M,:]),[N,M*tf.shape(X)[1]])
g_theta_et2 = g_theta_et1*tf.repeat(theta_et,M,axis=1)

g_theta_et = tf.transpose(tf.reshape((tf.reduce_sum(g_theta_et2,axis = 0, keepdims= True)),[tf.shape(X)[1],M]),perm=[1,0])
h_bs_c = tf.complex(X[2*N*M:2*N*M+M,:],X[2*N*M+M:2*N*M+2*M,:])

Es = etta*tau*P_b*(tf.real(tf.norm((h_bs_c+g_theta_et), axis=0))**2)
P_s = etta*(tau/(1-tau))*P_b*(tf.real(tf.norm((h_bs_c+g_theta_et), axis=0))**2)

g_theta_it1 = tf.complex(X[2*N*M+2*M:2*N*M+2*M+N,:],X[2*N*M+2*M+N:2*N*M+2*M+2*N,:])
g_theta_it = tf.reduce_sum(g_theta_it1*theta_it,axis = 0)
h_sd_c = tf.complex(X[2*N*M+2*M+2*N:2*N*M+2*M+2*N+1,:],X[2*N*M+2*M+2*N+1:2*N*M+2*M+2*N+2,:])

rd = P_s*(tf.abs((h_sd_c+g_theta_it)**2))/((sigma_n)**2)
r = (1-tau)*tf.log(1+rd)/tf.log(log_base)
cost = -tf.reduce_mean(r)


## Initialize global variables

In [50]:
init = tf.global_variables_initializer()

In [51]:

sess = tf.Session()
sess.run(init)

## Average Rate and Time for a batch

In [52]:
M_C =1000

mini_batch_size = 10**3
np.random.seed(112)    # Must be same seed as used in GA

h_bs_test = (np.random.randn( M,mini_batch_size)+1j*np.random.randn( M,mini_batch_size))*np.sqrt(b_bs/2)
h_is_test = (np.random.randn( 1,mini_batch_size)+1j*np.random.randn( 1,mini_batch_size))*np.sqrt(b_is/2)
h_id_test = (np.random.randn( 1,mini_batch_size)+1j*np.random.randn( 1,mini_batch_size))*np.sqrt(b_id/2)
h_sd_test = (np.random.randn( 1,mini_batch_size)+1j*np.random.randn( 1,mini_batch_size))*np.sqrt(b_sd/2)
g_rs_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_rs/2)
g_sr_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_rd/2)
g_rd_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_rd/2)
g_ir_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_ir/2)
G_br_test = (np.random.randn( N,M*mini_batch_size)+1j*np.random.randn( N,M*mini_batch_size))*np.sqrt(b_br/2)

tic = time.time()
g_rs1 =  np.repeat(g_rs_test,M,axis = 1)
F = np.reshape(G_br_test*g_rs1, [N*M,mini_batch_size])
F = np.concatenate([np.real(F),np.imag(F)],axis=0)
F1 = np.concatenate([np.real(h_bs_test),np.imag(h_bs_test)],axis=0)
F2 = g_rd_test*g_rs_test
F2 = np.concatenate([np.real(F2),np.imag(F2)],axis=0)
F3 = np.concatenate([np.real(h_sd_test),np.imag(h_sd_test)],axis=0)
F = np.concatenate([F,F1,F2,F3],axis=0)

rate = sess.run(cost, feed_dict={X:F})
toc = time.time() 
print('Time for a Batch of 1000 test data:', toc-tic)
print('Avg Rate:',rate)
tau1 = sess.run(tau, feed_dict={X:F})
print('Avg tau',np.mean(tau1))

Time for a Batch of 1000 test data: 0.14621210098266602
Avg Rate: -3.2593278875161706
Avg tau 0.31937924747394253


## Average Time taken by IRS-Net to compute Rate

In [53]:
mini_batch_size = 1
t = np.zeros((1000,1))
for i in range(1000):
    np.random.seed(i*2)
    h_bs_test = (np.random.randn( M,mini_batch_size)+1j*np.random.randn( M,mini_batch_size))*np.sqrt(b_bs/2)
    h_is_test = (np.random.randn( 1,mini_batch_size)+1j*np.random.randn( 1,mini_batch_size))*np.sqrt(b_is/2)
    h_id_test = (np.random.randn( 1,mini_batch_size)+1j*np.random.randn( 1,mini_batch_size))*np.sqrt(b_id/2)
    h_sd_test = (np.random.randn( 1,mini_batch_size)+1j*np.random.randn( 1,mini_batch_size))*np.sqrt(b_sd/2)
    g_rs_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_rs/2)
    g_sr_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_rd/2)
    g_rd_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_rd/2)
    g_ir_test = (np.random.randn( N,mini_batch_size)+1j*np.random.randn( N,mini_batch_size))*np.sqrt(b_ir/2)
    G_br_test = (np.random.randn( N,M*mini_batch_size)+1j*np.random.randn( N,M*mini_batch_size))*np.sqrt(b_br/2)
    
    tic = time.time()
    g_rs1 =  np.repeat(g_rs_test,M,axis = 1)
    F = np.reshape(G_br_test*g_rs1, [N*M,mini_batch_size])
    F = np.concatenate([np.real(F),np.imag(F)],axis=0)
    F1 = np.concatenate([np.real(h_bs_test),np.imag(h_bs_test)],axis=0)
    F2 = g_rd_test*g_rs_test
    F2 = np.concatenate([np.real(F2),np.imag(F2)],axis=0)
    F3 = np.concatenate([np.real(h_sd_test),np.imag(h_sd_test)],axis=0)
    F = np.concatenate([F,F1,F2,F3],axis=0)

    rate = sess.run(cost, feed_dict={X:F})
    toc = time.time() 
    t[i] = toc-tic
print('Average time per 1 channel Coefficient',np.mean(t))

Average time per 1 channel Coefficient 0.0006310133934020996
